# XGBoost

In [ ]:
import pandas as pd
%matplotlib inline

from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [ ]:
# download test data
df = pd.read_csv (r'C:\Users\Emelie\Documents\Modern method in ML\Project\project_train.csv')
# download train data that's given from canvas
X_testR = pd.read_csv (r'C:\Users\Emelie\Documents\Modern method in ML\Project\project_test.csv')

Take away outliers

In [ ]:
df = df.drop([68,94,84], axis=0)

In [ ]:
X = df.iloc[:, :-1].values
y = df.iloc[:, 11].values

# we split the training data into validation and training data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)


### Scaling the data 

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train) # scale against train data

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

scaler = StandardScaler()
scaler.fit(X) # scale against train data

# for cross validation and finished model
X_scaled = scaler.transform(X)
# Given test data from canvas 
X_scaledR = scaler.transform(X_testR)

In [ ]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
X_scaled = min_max_scaler.fit_transform(X)

### XGBoost Dart
The "Dart" model take away some features and trees

In [ ]:
xgb_clf = XGBClassifier(booster='dart',learning_rate=0.15,max_depth=6)

scores = cross_val_score(xgb_clf, X_scaled, y, cv=KFold(n_splits=10, random_state=1234, shuffle=True), scoring = 'accuracy')
print(np.mean(scores))
print(np.var(scores))
print(np.std(scores))

### Feature selection
Doesn't help! 

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2


xgb_clf = XGBClassifier(booster='dart',learning_rate=0.15,max_depth=6)
acc2 = []
var2 = []
features =  range(2, 11)
for i in features: 
    X_new = SelectKBest(chi2 , k=i).fit_transform(X_scaled, y)
    acc_temp = cross_val_score(xgb_clf, X_new, y, cv=KFold(n_splits=10, random_state=1234, shuffle=True))
    acc2.append(acc_temp.mean())
    var2.append(acc_temp.var())


plt.errorbar(features, acc2, yerr = var2) 
  
plt.title('Dimensionality reduction analysis') 
plt.ylabel('Accuracy')
plt.xlabel('K top features')
plt.show() 

In [ ]:
xgb_clf = XGBClassifier(booster='dart',learning_rate=0.15,max_depth=6)
X_new = SelectKBest(chi2 , k=7).fit_transform(X_scaled, y)
acc = cross_val_score(xgb_clf, X_new, y, cv=KFold(n_splits=10, random_state=1234, shuffle=True))
print("Accuracy: ", acc.mean(), "(+/-",  acc.var(),")")

#### Gridsearch

In [ ]:
model = XGBClassifier()
clf = GridSearchCV(model,
                  {'booster':['dart'], 'learning_rate':[0.15],
                   'max_depth':[6], 'n_estimators': [20,55,80], 
                   'n_jobs':[1,10,20], 'num_parallel_tree': [1,5,10] })
clf.fit(X_train, y_train)
print('Best params for DART')
print(clf.best_score_)
print(clf.best_params_)

### Run model on canvas data

In [ ]:
xgb_clf = XGBClassifier(booster='dart',learning_rate=0.15,max_depth=6)

xgb_clf.fit(X_scaled, y)
ypred = xgb_clf.predict(X_scaledR)
print(ypred)
#make it to a dataframe
df = pd.DataFrame(ypred)
#save to xlsx file
filepath = 'result_10.xlsx'.format()
#Save to computer
df.to_excel(filepath, index=False)